In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# 1. Hugging Face 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")  # 예제 토크나이저, 원하는 모델로 변경 가능
EOS_TOKEN = tokenizer.eos_token  # EOS 토큰 추가

# 2. Alpaca 스타일 프롬프트 템플릿 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Glossary:
{}

### Response:
{}"""  # 새로운 템플릿 적용

# 3. 엑셀 파일을 데이터프레임으로 로드
df = pd.read_excel("datasetF.xlsx")  # 엑셀 파일 경로
# ✅ 컬럼명을 소문자로 변환하고 공백 제거 (자동화)
df.columns = df.columns.str.lower().str.strip()

# 4. Alpaca 스타일 프롬프트 적용 (새로운 컬럼 추가)
def apply_prompt_template(row):
    glossary_text = row["metadata"] if isinstance(row["metadata"], str) else "N/A"
    return alpaca_prompt.format(row["instruction"], row["input"], glossary_text, row["output"]) + EOS_TOKEN
# 'text' 컬럼 추가
df["text"] = df.apply(apply_prompt_template, axis=1)

# 5. 데이터프레임을 트레인셋과 테스트셋으로 분할 (80:20 비율)
train_df = df.sample(frac=0.8, random_state=42)  # 80%
test_df = df.drop(train_df.index)  # 나머지 20%

# 6. 인덱스 컬럼 제거 (필요 없으면 drop)
train_df = train_df.reset_index(drop=True)  # 인덱스를 새로 생성하고 기존 인덱스 제거
test_df = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df).map(batched=True)
test_dataset = Dataset.from_pandas(test_df).map(batched=True)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})


# 7. 데이터셋을 DatasetDict로 묶기
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# 8. 결과 출력
print(dataset)


Map:   0%|          | 0/21662 [00:00<?, ? examples/s]

Map:   0%|          | 0/5416 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'target language', 'metadata', 'text'],
        num_rows: 21662
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'target language', 'metadata', 'text'],
        num_rows: 5416
    })
})


In [24]:
import os

# 9. Hugging Face 데이터셋 공간을 만들고 Parquet 파일 업로드
hf_token = os.getenv('HUGGING_FACE_TOKEN')  # 보안 강화를 위해 입력받음
login(token=hf_token)  # Hugging Face 로그인

dataset.push_to_hub("haeun0420/gorani_full_finetuning_datasets")  # 데이터셋 업로드

print("✅ Alpaca 템플릿 적용 완료! EOS_TOKEN 추가 완료! Parquet 변환 및 Hugging Face 업로드 완료!")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Alpaca 템플릿 적용 완료! EOS_TOKEN 추가 완료! Parquet 변환 및 Hugging Face 업로드 완료!
